Install the required libraries

In [ ]:
# !pip install tensorflow==2.3.0
# !pip install gym
!pip install keras
!pip install keras-rl2
!pip install 'gym[classic_control]'

Import the necessary libraries and set up the CartPole environment

In [2]:
import gym
import random
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation
from tensorflow.keras.optimizers import Adam
import os
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [3]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
actions = env.action_space.n

/usr/local/lib/python3.8/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
actions

2

In [5]:

os.environ['SDL_VIDEODRIVER']='dummy'

In [6]:
episodes = 20
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

/usr/local/lib/python3.8/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Episode:1 Score:14.0
Episode:2 Score:23.0
Episode:3 Score:21.0
Episode:4 Score:21.0
Episode:5 Score:16.0
Episode:6 Score:105.0
Episode:7 Score:12.0
Episode:8 Score:58.0
Episode:9 Score:15.0
Episode:10 Score:15.0
Episode:11 Score:14.0
Episode:12 Score:27.0
Episode:13 Score:19.0
Episode:14 Score:13.0
Episode:15 Score:19.0
Episode:16 Score:31.0
Episode:17 Score:25.0
Episode:18 Score:16.0
Episode:19 Score:29.0
Episode:20 Score:17.0


Define the model

Next, define a function that will create the model. 
This model will take in the current state of the environment as input and output the action to be taken.

In [7]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [8]:
model = build_model(states, actions)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 32)                160       
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 1,282
Trainable params: 1,282
Non-trainable params: 0
_________________________________________________________________


Build the agent and train the model

In [10]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [11]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=True, verbose=1)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training for 50000 steps ...
Interval 1 (0 steps performed)
    4/10000 [..............................] - ETA: 3:23 - reward: 1.0000 

/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/usr/local/lib/python3.8/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


   10/10000 [..............................] - ETA: 3:21 - reward: 1.0000

/usr/local/lib/python3.8/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   21/10000 [..............................] - ETA: 8:59 - reward: 1.0000

/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 15 + 1) instead
  batch_idxs

   30/10000 [..............................] - ETA: 7:16 - reward: 1.0000

/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 22 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 23 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 24 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 25 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.8/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 26 + 1) instead
  batch_idxs

10000/10000 [==============================] - 203s 20ms/step - reward: 1.0000
82 episodes - episode_reward: 120.902 [8.000, 500.000] - loss: 2.475 - mae: 21.068 - mean_q: 42.770

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 202s 20ms/step - reward: 1.0000
30 episodes - episode_reward: 328.533 [257.000, 500.000] - loss: 4.775 - mae: 46.245 - mean_q: 93.583

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 202s 20ms/step - reward: 1.0000
28 episodes - episode_reward: 349.393 [279.000, 500.000] - loss: 4.910 - mae: 52.077 - mean_q: 105.062

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 202s 20ms/step - reward: 1.0000
25 episodes - episode_reward: 416.800 [315.000, 500.000] - loss: 4.627 - mae: 53.337 - mean_q: 107.454

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 202s 20ms/step - reward: 1.0000
done, took 1011.950 seconds


In [12]:
scores = dqn.test(env, nb_episodes=100, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 337.000, steps: 337
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 400.000, steps: 400
Episode 4: reward: 430.000, steps: 430
Episode 5: reward: 438.000, steps: 438
Episode 6: reward: 443.000, steps: 443
Episode 7: reward: 452.000, steps: 452
Episode 8: reward: 410.000, steps: 410
Episode 9: reward: 366.000, steps: 366
Episode 10: reward: 452.000, steps: 452
Episode 11: reward: 431.000, steps: 431
Episode 12: reward: 384.000, steps: 384
Episode 13: reward: 389.000, steps: 389
Episode 14: reward: 359.000, steps: 359
Episode 15: reward: 420.000, steps: 420
Episode 16: reward: 364.000, steps: 364
Episode 17: reward: 448.000, steps: 448
Episode 18: reward: 344.000, steps: 344
Episode 19: reward: 431.000, steps: 431
Episode 20: reward: 372.000, steps: 372
Episode 21: reward: 344.000, steps: 344
Episode 22: reward: 467.000, steps: 467
Episode 23: reward: 468.000, steps: 468
Episode 24: reward: 452.000, steps: 452
Episode 25: reward: 

In [13]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 462.000, steps: 462
Episode 2: reward: 394.000, steps: 394
Episode 3: reward: 489.000, steps: 489
Episode 4: reward: 472.000, steps: 472
Episode 5: reward: 500.000, steps: 500
Episode 6: reward: 500.000, steps: 500
Episode 7: reward: 422.000, steps: 422
Episode 8: reward: 500.000, steps: 500
Episode 9: reward: 466.000, steps: 466
Episode 10: reward: 463.000, steps: 463
Episode 11: reward: 385.000, steps: 385
Episode 12: reward: 352.000, steps: 352
Episode 13: reward: 418.000, steps: 418
Episode 14: reward: 387.000, steps: 387
Episode 15: reward: 475.000, steps: 475


In [14]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [15]:
del model
del dqn
del env

Test the trained model

Now that I have trained the model, I can test it by running it on the CartPole environment and see how well it performs.

In [16]:
env = gym.make('CartPole-v1')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [17]:
dqn.load_weights('dqn_weights.h5f')

In [18]:
_ = dqn.test(env, nb_episodes=10, visualize=True)

Testing for 10 episodes ...
Episode 1: reward: 372.000, steps: 372
Episode 2: reward: 385.000, steps: 385
Episode 3: reward: 457.000, steps: 457
Episode 4: reward: 365.000, steps: 365
Episode 5: reward: 400.000, steps: 400
Episode 6: reward: 458.000, steps: 458
Episode 7: reward: 404.000, steps: 404
Episode 8: reward: 406.000, steps: 406
Episode 9: reward: 390.000, steps: 390
Episode 10: reward: 463.000, steps: 463
